# PyTorch Text Classifier

This notebook shows how to export a 🤗transformer model, upload it to easytensor, and query it on the cloud.

In [ ]:
! pip install easytensor transformers

# Training

🤗transformers make training a model rather easy but utilizing pre-trained models and allowing us to fine-tune them.


In [ ]:
from transformers import MobileBertTokenizer, MobileBertModel

model = MobileBertModel.from_pretrained("google/mobilebert-uncased")

# Tokenizers

We can use 🤗 pre-trained tokenizers to preprocess our input and feed it directly into the model!

Take note of the expansion of the input when it's passed to the model inference.

In [ ]:
tokenizer = MobileBertTokenizer.from_pretrained("google/mobilebert-uncased")
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

# Uploading to EasyTensor

Now that we have our model, We can upload it to EasyTensor. First, we create our model class definition and store it in a file

In [ ]:
model_definition = """
# This is our model file.
from transformers import MobileBertTokenizer, MobileBertModel, MobileBertConfig

class MyModel(MobileBertModel):
    def __init__(self, *args, **kwargs):
        MobileBertModel.__init__(self, *args, **kwargs)
        self.tokenizer = MobileBertTokenizer.from_pretrained(
            "google/mobilebert-uncased"
        )

    def predict_single(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        outputs = self(**inputs)
        return outputs[0].detach().numpy()[0].tolist()
"""
with open("model_file.py", "w") as fout:
    fout.write(model_definition)

In [ ]:
import easytensor
model_id, access_token = easytensor.transformers.upload_model("test_hf", model, "model_file.py")
print("model id:", model_id, "access code:", access_token)

# Running Inference

With the model uploaded to EasyTensor, you're now ready to run inference on the model!

In [ ]:
import requests
response = requests.post(
        "https://app.easytensor.com/query/",
        json={
            "instances": [
                "This is a test"
            ]
        },
        headers={"accessToken": access_token}
    )
response.raise_for_status()
predictions = response.json()
print(predictions)